# This script processes the CDHit files and generates a csv with peptides and read counts for each peptide.
#Works around CDHit's issue of selecting a sequence randomly as the representative sequence.
#Use CDHit file which now has the sequence as part of the name

In [1]:
import numpy as np
import pandas as pd
import csv
import string
import matplotlib.pyplot as plt
import seaborn as sns
import filecmp
from scipy import stats
import re

In [2]:
codon_2_aa = {"TTT":"F", "TTC":"F", "TTA":"L", "TTG":"L",
    "TCT":"S", "TCC":"S", "TCA":"S", "TCG":"S",
    "TAT":"Y", "TAC":"Y", "TAA":"*", "TAG":"*",
    "TGT":"C", "TGC":"C", "TGG":"W", "TGA":"*",
    "CTT":"L", "CTC":"L", "CTA":"L", "CTG":"L",
    "CCT":"P", "CCC":"P", "CCA":"P", "CCG":"P",
    "CAT":"H", "CAC":"H", "CAA":"Q", "CAG":"Q",
    "CGT":"R", "CGC":"R", "CGA":"R", "CGG":"R",
    "ATT":"I", "ATC":"I", "ATA":"I", "ATG":"M",
    "ACT":"T", "ACC":"T", "ACA":"T", "ACG":"T",
    "AAT":"N", "AAC":"N", "AAA":"K", "AAG":"K",
    "AGT":"S", "AGC":"S", "AGA":"R", "AGG":"R",
    "GTT":"V", "GTC":"V", "GTA":"V", "GTG":"V",
    "GCT":"A", "GCC":"A", "GCA":"A", "GCG":"A",
    "GAT":"D", "GAC":"D", "GAA":"E", "GAG":"E",
    "GGT":"G", "GGC":"G", "GGA":"G", "GGG":"G"}

def translate(pep_dna):
#Translate peptide to amino acids
    codons_aa = []
    codons_dna = []
    for j in range(0,len(pep_dna),3):   
        codons_dna.append(pep_dna[j:j+3])
    codons_aa = [codon_2_aa[x] for x in codons_dna] # translate
    pep_aa=''.join(codons_aa) #puts letters into string
    return pep_aa

In [3]:
def bad_clusters_fn(file_name):
#Identify which cluster numbers have incorrectly selected a representative sequence.
#goes through .clstr file
#calculates the percent of members of each cluster with sequence identity at 100% or other percent
#outputs the cluster number and the number of total peptides and the number of peptides that match the representative sequence
    total_list=[]
    matches_list=[]
    cluster_list=[]

    matches = 0
    total = 0

    with open(file_name, 'r') as read_file:  
        for line in read_file:
            if line[0] == '>':
                cluster = line[line.strip().find('Cluster ')+len('Cluster '):-1]
                cluster_list.append(cluster)
                matches_list.append(matches+1)#add 1 because the rep seq is at 100%
                total_list.append(total)
                matches = 0
                total = 0
            else:
                total+=1
                if '%' in line:
                    percent = line[line.find('+/')+len('+/'):line.rfind('%')]
                    if percent == '100.00':
                        matches+=1
        matches_list.append(matches+1)#add 1 because the rep seq is at 100%
        total_list.append(total)             

    clusters = pd.DataFrame({
        'cluster#':cluster_list,
        'perfect_match':matches_list[1:],
        'total_seq':total_list[1:]})

    clusters['frac_100'] = clusters['perfect_match']/clusters['total_seq']

    #select subset of clusters with fraction of members with perfect sequence identity to representative sequence below 50%
    bad_clusters = clusters[ clusters['frac_100']< 0.50 ]   
    print('Number of clusters with incorrect representative sequence assigned by CDHit:',len(bad_clusters))
    #bad_clusters.to_csv(roundx+'_badclusters.csv', index=False)
    return bad_clusters

def bad_cluster_members_fn(file_name, bad_clusters):
# Go through CDhit file and pull out the sequence names corresponding to clusters 
## that have a representative sequence which doesn't match well to the other sequences
    cluster_list = []
    sequence_name_list = []
    seq_list = []
    percent_list = []
    the_clust_flag = 0

    i=0
    with open(file_name, 'r') as read_file:  
        for line in read_file:
            #check if this cluster number is in the list of 'offending' clusters
            if line[0]=='>':
                line_cluster = line.strip()[line.strip().find('Cluster ')+len('Cluster '):] 
                if i<len(bad_clusters):
                    if line_cluster==bad_clusters['cluster#'].iloc[i]:
                        the_clust_flag = 1
                        i+=1
                    else:
                        the_clust_flag = 0
                else:
                    the_clust_flag = 0
            if line[0]!='>' and the_clust_flag==1:
                cluster_list.append(line_cluster)
                
                sequence_name=line[line.find('39nt, ')+len('39nt, '):line.find('...')]
                sequence_name_list.append(sequence_name.strip())
                seq_list.append(sequence_name.strip().split('_')[1])

                percent = line.strip()[line.strip().find('+/')+len('+/'):line.strip().find('%')]
                if line.strip()[-1]=='*':
                    percent=float(100)
                percent_list.append(float(percent))

            if i>len(bad_clusters):
                break
    
    bugged = pd.DataFrame({
        'cluster_num':cluster_list,
        'seq_id':sequence_name_list,
        'percent_id':percent_list,
        'dna':seq_list
        })

    bad_cluster_members = bugged
    return bad_cluster_members


def count_fn(bad_cluster_members):
#count number of times that DNA showed up in a cluster and the number of sequences in the cluster
    #count # of times that DNA seq showed up in bad_cluster_members
    dna_count = bad_cluster_members.groupby('dna')['seq_id'].nunique()
    dna_count_df = pd.DataFrame({'dna':dna_count.index, 'count_dna':dna_count.values}) #makes into a dataframe
    bugged_dna_count = pd.merge(dna_count_df, bad_cluster_members, on=['dna']) #was df_counts

    #count # of sequences in that cluster
    cluster_count = bugged_dna_count.groupby('cluster_num')['seq_id'].nunique()
    cluster_count_df = pd.DataFrame({'cluster_num':cluster_count.index, 'count_clust':cluster_count.values}) #makes into a dataframe

    bad_cluster_members_count = pd.merge(cluster_count_df, bugged_dna_count, on=['cluster_num'])
    bad_cluster_members_count['frac'] = bad_cluster_members_count['count_dna']/bad_cluster_members_count['count_clust']
    return bad_cluster_members_count

def replace_list_fn(bad_cluster_members_count):
    old_list=[]
    new_list=[]

    for x in bad_cluster_members_count['cluster_num'].unique():
        y = bad_cluster_members_count[bad_cluster_members_count['cluster_num'] == x]
        old = y['dna'].loc[y['percent_id'].idxmax()] #old sequence has 100% identity to representative seq (is the old rep seq)
        old_list.append(old)
        new = y['dna'].loc[y['count_dna'].idxmax()] #take sequence with max number of sequences like it
        new_list.append(new)

    replacement_df = pd.DataFrame({'old_rep':old_list, 'new_rep':new_list})
    
    return replacement_df


#calls other functions for debugging CDHit
def fix_cdhit_bug(file_name):
    file_name_clstr = file_name + '.clstr'
    bad_clusters = bad_clusters_fn(file_name_clstr)
    bad_cluster_members = bad_cluster_members_fn(file_name_clstr, bad_clusters) 
    bad_cluster_members_count = count_fn(bad_cluster_members)
    replacement_df = replace_list_fn(bad_cluster_members_count)
    return replacement_df

In [4]:
def import_cdhit_rep(file_name, replacement_df):
#Import sequences from CD-Hit representative-sequences file
##In sets of twos, lines contain [Name, Sequence].  Import every second line (not the name line)
    read_list = []
    with open(file_name, 'r') as read_file:  
        for line in read_file:
            if '>' not in line:
                #check if line is in offenders list for wrong representative sequence chosen by CDHit
                if line.strip() in replacement_df['old_rep'].values:
                    new = replacement_df[replacement_df['old_rep'].isin([line.strip()])]['new_rep'].values
                    read_list.append(translate(new.item()))
                else:
                    read_list.append(translate(line.strip('\n')))
                    
    return read_list

def import_cdhit_clstr(file_name):
#Import sequences from CD-Hit cluster file
    #
    ##The file contains a list of clusters of the format:
    ### >Cluster X
    ### 0       39nt, >1101:14633:1491:ATC... *
    ### 1       39nt, >1101:12909:1680:ATC... at +/100.00%
    ### etc
    #Look for the '>' flag for the new cluster then extract the number of sequences in the previous cluster
    #
    clstr_file_name = file_name + '.clstr' #file containing clusters
    cluster_count = 0
    first_line_flag = 1
    count_list = [] 
    with open(clstr_file_name, 'r') as read_file:  
        for line in read_file:
            #the line preceding a new cluster tells the number of sequences in that previous cluster
            if first_line_flag==1:
                first_line_flag=0
                line_before=line
            else:
                if line[0] == '>': 
                    count_list.append( int(line_before.split('\t')[0])+1 ) #start at 0, so add 1 to cluster number label

                    cluster_count=cluster_count+1
                line_before=line
        #Last line
        count_list.append( int(line_before.split('\t')[0])+1 )
        cluster_count=cluster_count+1
        #print("Number of unique DNA sequences (with >1 nucleotide difference):", cluster_count)
    return count_list



In [5]:
def confirm_order(rep_file_name):
# Confirm that the representative sequences in order match the representative sequences of the clusters
# CD-Hit documentation does not explicitly confirm
    with open(rep_file_name+'.clstr', 'r') as read_file, open('rep_check', 'w', newline='') as write_file:
        for line in read_file:
            if '*' in line:
                pep=line.split('>')[-1].rsplit(':',1)[0] #take sequence between the '>' and the last ':'
                write_file.write(pep+ '\n')
    with open(rep_file_name, 'r') as read_file, open('clstr_check', 'w', newline='') as write_file:
        for line in read_file:
            if '>' in line:
                pep=line.split('>')[-1].rsplit(':',1)[0] 
                write_file.write(pep+ '\n')
    if filecmp.cmp('rep_check', 'clstr_check')==False: # filecmp returns true if they seem equal
        print('ERROR: Representative sequence order incorrect')
    else:
        print('Proceed: Representative sequence order correct')
        
def collapse2uniqAA(illumina):
#Some DNA sequences are unique but, when translated, are non-unique AA sequences. Combine non-unique AA sequences(e.g. ALPWRSVWL)
    illumina_nodup=illumina.groupby(['sequence']).sum().reset_index()  
    if np.sum(illumina_nodup['count'])==np.sum(illumina['count']):
        print('Proceed: Number of reads unchanged while collapsing AA sequence duplicates')
    else:
        print('Error: Number of reads changed while collapsing AA duplicates')
    return illumina_nodup


In [6]:
def import_cdhit_all( roundx):
    print( roundx)
    rep_file_name = 'pepdist_{}_cdhit'.format(roundx)
    
    replacement_df = fix_cdhit_bug(rep_file_name)
    read_list = import_cdhit_rep(rep_file_name, replacement_df)
    count_list = import_cdhit_clstr(rep_file_name)
    confirm_order(rep_file_name)
    
    #Combine sequences and read count for each sequence
    illumina = pd.DataFrame(
    {'sequence': read_list,
     #'dna': dna_list,
     'count': count_list,
    })
    
    illumina_nodup = collapse2uniqAA(illumina)
    
    return illumina_nodup


for roundx in ["DR401_R1","DR401_R2","DR401_R3"]:
    illumina_nodup = import_cdhit_all( roundx)
    illumina_nodup.to_csv('{}_cdhit-corrected_data.csv'.format(roundx))

DR401_R1
Number of clusters with incorrect representative sequence assigned by CDHit: 116
Proceed: Representative sequence order correct
Proceed: Number of reads unchanged while collapsing AA sequence duplicates
DR401_R2
Number of clusters with incorrect representative sequence assigned by CDHit: 193
Proceed: Representative sequence order correct
Proceed: Number of reads unchanged while collapsing AA sequence duplicates
DR401_R3
Number of clusters with incorrect representative sequence assigned by CDHit: 252
Proceed: Representative sequence order correct
Proceed: Number of reads unchanged while collapsing AA sequence duplicates
